<a href="https://colab.research.google.com/github/efkomarova/Sirius/blob/main/Behavioral_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Поведенческие паттерны (Behavioral)

часть 3: **как объекты взаимодействуют**.

- **Chain of Responsibility** — запрос по цепочке
- **Command** — действие как объект
- **Interpreter** — выражение = дерево
- **Iterator** — hasNext/next
- **Mediator** — все через посредника
- **Memento** — снимок состояния
- **Observer** — подписка и рассылка
- **State** — поведение от состояния
- **Strategy** — алгоритм подставляется
- **Template Method** — скелет + шаги
- **Visitor** — операция снаружи

## 13. Chain of Responsibility — по цепочке

**Легенда:** Спросил папу — не знает. Папа передаёт маме. Вопрос идёт по цепочке, пока кто-то не обработает.

**Зачем нужен:** Несколько уровней обработки. Техподдержка, одобрение заявок по уровням. Каждый знает «следующего», решает: моё — обработал, не моё — передал.

In [7]:
%%writefile chain_of_responsibility.cpp
/* ЛЕГЕНДА: Вопрос — спросил одного, не знает. Спросил другого. По цепочке. СУТЬ: запрос передаётся, пока кто-то не обработает. */

#include <iostream>
using namespace std;

// Базовый класс — каждый «отвечающий» знает про следующего в цепочке
class Answerer {
protected:
    Answerer* next = nullptr;  // следующий в цепочке
public:
    void setNext(Answerer* m) { next = m; }
    virtual void handle(const string& q) {
        if (next)
            next->handle(q);   // не знаю — передаю дальше
        else
            cout << "Не знают\n";  // конец цепочки
    }
};

// Папа — обрабатывает только «математика»
class Dad : public Answerer {
public:
    void handle(const string& m) override {
        if (m == "математика")
            cout << "Папа ответил\n";
        else
            Answerer::handle(m);  // не моё — передаю next (маме)
    }
};

// Мама — обрабатывает «готовка»
class Mom : public Answerer {
public:
    void handle(const string& m) override {
        if (m == "готовка")
            cout << "Мама ответила\n";
        else
            Answerer::handle(m);
    }
};

int main() {
    Dad d;
    Mom m;
    d.setNext(&m);  // цепочка: папа -> мама
    d.handle("математика");  // папа обработает
    d.handle("готовка");     // папа передаст маме
}

Writing chain_of_responsibility.cpp


In [8]:
!g++ -std=c++17 -o out chain_of_responsibility.cpp && ./out

Папа ответил
Мама ответила


## 14. Command — действие как объект

**Легенда:** Заметка «купить молоко». Записал — объект «команда» существует. Выполнишь потом. Или отменишь.

**Зачем нужен:** Откладывать выполнение, ставить в очередь, Undo. Если действие — объект с `execute()`, его можно сохранить, передать, выполнить позже.

In [9]:
%%writefile command.cpp
/* ЛЕГЕНДА: Заметка «купить молоко» — записал. Сделаешь потом. СУТЬ: действие как объект — можно хранить, отложить, отменить. */

#include <iostream>
using namespace std;

// Команда — действие, упакованное в объект. Можно положить в список, выполнить позже
class Command {
public:
    virtual void execute() = 0;  // выполнить команду (сейчас или позже)
    virtual ~Command() = default;
};

// Получатель — объект, над которым выполняется действие
class ShoppingList {
public:
    void add(const string& item) { cout << "Добавлено: " << item << endl; }
};

// Конкретная команда — «добавить молоко в список»
class NoteCommand : public Command {
    ShoppingList* list;  // на какой список действуем
    string item;         // что добавить
public:
    NoteCommand(ShoppingList* l, const string& i) : list(l), item(i) {}
    void execute() override {
        list->add(item);  // выполняем действие (можно было бы отложить)
    }
};

int main() {
    ShoppingList list;
    Command* cmd = new NoteCommand(&list, "молоко");  // создали — пока не выполнили
    cmd->execute();  // можно было сохранить cmd и вызвать execute() вечером
    delete cmd;
}

Overwriting command.cpp


In [10]:
!g++ -std=c++17 -o out command.cpp && ./out

Добавлено: молоко


## 15. Interpreter — выражение как дерево

**Легенда:** Калькулятор. 4+3 — дерево: плюс(4, 3). Каждый узел умеет себя вычислить.

**Зачем нужен:** Выражения, формулы. Дерево объектов. Каждый тип узла — класс с `eval()`. Добавить операцию — новый класс.

In [11]:
%%writefile interpreter.cpp
/* ЛЕГЕНДА: Калькулятор — 4+3. Считает по дереву: плюс(4, 3). СУТЬ: выражение = дерево объектов, каждый узел вычисляет себя. */

#include <iostream>
using namespace std;

// Базовый класс выражения. Каждый узел дерева — Expr
class Expr {
public:
    virtual int eval() = 0;  // вычислить значение (рекурсивно)
    virtual ~Expr() = default;
};

// Число — лист дерева. Просто возвращает себя
class Num : public Expr {
    int val;
public:
    Num(int v) : val(v) {}
    int eval() override { return val; }
};

// Сложение — ветка. Два подвыражения (левое и правое)
class Add : public Expr {
    Expr* left, *right;
public:
    Add(Expr* l, Expr* r) : left(l), right(r) {}
    int eval() override {
        return left->eval() + right->eval();  // рекурсия: сначала вычислить детей
    }
};

int main() {
    // Дерево: Add(Num(4), Num(3))  — плюс с аргументами 4 и 3
    Expr* e = new Add(new Num(4), new Num(3));
    cout << e->eval() << endl;  // 7
}

Overwriting interpreter.cpp


In [12]:
!g++ -std=c++17 -o out interpreter.cpp && ./out

7


## 16. Iterator — обход без знаний

**Легенда:** Книга. Листаешь страницы. Не знаешь, как они хранятся. Просто: есть следующая? Дай.

**Зачем нужен:** Скрыть устройство коллекции. Код перебора одинаков. Меняешь структуру — итератор меняешь, перебор тот же.

In [13]:
%%writefile iterator.cpp
/* ЛЕГЕНДА: Книга — листаешь страницы. Следующая, следующая... СУТЬ: обход коллекции без знания её внутреннего устройства. */

#include <iostream>
#include <vector>
using namespace std;

// template<typename T> — итератор для любого типа T (страницы, числа, объекты)
template<typename T>
class PageIterator {
    vector<T>& pages;  // ссылка на вектор (не копия! & важно)
    int i = 0;         // текущая позиция
public:
    PageIterator(vector<T>& v) : pages(v) {}
    bool hasNext() { return i < (int)pages.size(); }  // есть ли следующий?
    T next() { return pages[i++]; }  // взять текущий и сдвинуться (i++ после)
};

int main() {
    vector<string> book = {"Стр. 1", "Стр. 2", "Стр. 3"};
    PageIterator<string> it(book);
    while (it.hasNext())
        cout << it.next() << " ";
    cout << endl;
}

Overwriting iterator.cpp


In [14]:
!g++ -std=c++17 -o out iterator.cpp && ./out

Стр. 1 Стр. 2 Стр. 3 


## 17. Mediator — все через одного

**Легенда:** Учитель в классе. Дети спрашивают его, не друг друга. Все общаются через учителя.

**Зачем нужен:** Много объектов должны общаться. Без посредника — десятки связей. Посредник знает всех, рассылает сообщения.

In [15]:
%%writefile mediator.cpp
/* ЛЕГЕНДА: Учитель в классе — дети спрашивают его, не друг друга. СУТЬ: центральный посредник, все общаются через него. */

#include <iostream>
#include <vector>
using namespace std;

class Student;  // предварительное объявление (Teacher использует Student)

// Посредник — знает всех учеников, рассылает сообщения
class Teacher {
    vector<Student*> students;
public:
    void add(Student* s);
    void broadcast(Student* from, const string& msg);  // «передай от from всем»
};

// Участник — общается только через учителя, не знает других учеников
class Student {
    string name;
    Teacher* teacher;
public:
    Student(const string& n, Teacher* t) : name(n), teacher(t) {}
    void ask(const string& msg) { teacher->broadcast(this, msg); }  // не пишет каждому!
    string getName() { return name; }
    virtual void hear(const string& from, const string& msg) {
        cout << name << " слышит от " << from << ": " << msg << endl;
    }
};

// Реализация методов Teacher (определены вне класса)
void Teacher::add(Student* s) { students.push_back(s); }
void Teacher::broadcast(Student* from, const string& msg) {
    for (auto s : students)
        if (s != from)  // кроме того, кто спросил
            s->hear(from->getName(), msg);
}

int main() {
    Teacher t;
    Student s1("Аня", &t), s2("Ваня", &t);
    t.add(&s1); t.add(&s2);
    s1.ask("Сколько 2+2?");  // Аня -> учитель -> Ваня слышит
}

Writing mediator.cpp


In [16]:
!g++ -std=c++17 -o out mediator.cpp && ./out

Ваня слышит от Аня: Сколько 2+2?


## 18. Memento — снимок

**Легенда:** Игра. F5 — сохранился. Погиб. F9 — загрузился с checkpoint. Состояние «сфотографировано» в объект.

**Зачем нужен:** Undo, откат. Сохраняем копию полей в отдельный объект (Memento). Восстановление — копируем обратно.

In [17]:
%%writefile memento.cpp
/* ЛЕГЕНДА: Сохранение в игре — погиб, загрузился с checkpoint. СУТЬ: сохраняет состояние объекта, чтобы потом восстановить. */

#include <iostream>
using namespace std;

// Снимок (Memento) — хранит копию состояния. Неизменяемый «пакет»
// friend class Game — только Game может читать/писать приватные поля
class Checkpoint {
    int level;
    int score;
    friend class Game;
public:
    Checkpoint() : level(0), score(0) {}
};

// Игра — создаёт снимки и восстанавливается из них
class Game {
    int level = 1;
    int score = 0;
public:
    Checkpoint save() {
        Checkpoint cp;
        cp.level = level;   // сохраняем копию состояния
        cp.score = score;
        return cp;
    }
    void load(const Checkpoint& cp) {
        level = cp.level;   // восстанавливаем из снимка
        score = cp.score;
    }
    void advance() { level++; score += 100; }
    void show() { cout << "Уровень " << level << ", очки " << score << endl; }
};

int main() {
    Game g;
    g.show();
    auto cp = g.save();   // сохранились (level=1, score=0)
    g.advance();
    g.advance();
    g.show();             // level=3, score=200
    g.load(cp);           // откат к сохранению
    g.show();             // снова level=1, score=0
}

Writing memento.cpp


In [18]:
!g++ -std=c++17 -o out memento.cpp && ./out

Уровень 1, очки 0
Уровень 3, очки 200
Уровень 1, очки 0


## 19. Observer — подписка

**Легенда:** Подписка на канал. Вышел пост — всем пришло уведомление. Подписался — получаешь.

**Зачем нужен:** Один объект меняется — несколько других должны отреагировать. Источник хранит список подписчиков, при изменении вызывает `update()`.

In [19]:
%%writefile observer.cpp
/* ЛЕГЕНДА: Подписка — вышел пост, всем пришёл. СУТЬ: при изменении одного объекта все подписчики уведомляются. */

#include <iostream>
#include <vector>
using namespace std;

// Подписчик — любой, кто хочет получать уведомления. update() вызовется при новости
class Subscriber {
public:
    virtual void update(const string& news) = 0;
    virtual ~Subscriber() = default;
};

// Источник (Subject) — рассылает всем при publish()
class NewsLetter {
    vector<Subscriber*> subs;  // список подписчиков
public:
    void subscribe(Subscriber* s) { subs.push_back(s); }
    void publish(const string& news) {
        for (auto s : subs)
            s->update(news);  // уведомляем каждого
    }
};

// Читатель — конкретный подписчик
class Reader : public Subscriber {
    string name;
public:
    Reader(const string& n) : name(n) {}
    void update(const string& news) override {
        cout << name << ": " << news << endl;
    }
};

int main() {
    NewsLetter nl;
    Reader r1("Анна"), r2("Борис");
    nl.subscribe(&r1);  // & — адрес переменной (указатель)
    nl.subscribe(&r2);
    nl.publish("Вышел новый выпуск!");  // оба получат уведомление
}

Writing observer.cpp


In [20]:
!g++ -std=c++17 -o out observer.cpp && ./out

Анна: Вышел новый выпуск!
Борис: Вышел новый выпуск!


## 20. State — поведение от состояния

**Легенда:** Кран. Закрыт — не течёт. Открыл — течёт. Разные действия в зависимости от состояния.

**Зачем нужен:** Объект ведёт себя по-разному в разных режимах. Каждое состояние — отдельный класс. Вместо `if (state==1)...` во всех методах.

In [21]:
%%writefile state.cpp
/* ЛЕГЕНДА: Кран — закрыт не течёт, открыл течёт. СУТЬ: поведение зависит от внутреннего состояния. */

#include <iostream>
using namespace std;

class Faucet;  // предварительное объявление (FaucetState использует Faucet)

// Состояние — знает, что делать при действиях. Каждый класс = одно состояние
class FaucetState {
public:
    virtual void turnOn(Faucet* f) = 0;
    virtual void turnOff(Faucet* f) = 0;
};

// Контекст — хранит текущее состояние и делегирует ему все действия
class Faucet {
    FaucetState* state;
public:
    void setState(FaucetState* s) { state = s; }
    void on()  { state->turnOn(this); }   // this — указатель на себя
    void off() { state->turnOff(this); }
};

// Состояние «закрыт»
class Closed : public FaucetState {
public:
    void turnOn(Faucet* f) override;  // реализация ниже (нужен класс Open)
    void turnOff(Faucet*) override { cout << "Уже закрыт\n"; }
};

// Состояние «открыт»
class Open : public FaucetState {
public:
    void turnOn(Faucet*) override { cout << "Уже открыт\n"; }
    void turnOff(Faucet* f) override {
        cout << "Вода перекрыта\n";
        f->setState(new Closed());  // переход в другое состояние!
    }
};

void Closed::turnOn(Faucet* f) {
    cout << "Вода течёт\n";
    f->setState(new Open());  // переход: Closed -> Open
}

int main() {
    Faucet faucet;
    faucet.setState(new Closed());  // начальное состояние
    faucet.on();   // Closed -> Open
    faucet.off();  // Open -> Closed
}

Writing state.cpp


In [22]:
!g++ -std=c++17 -o out state.cpp && ./out

Вода течёт
Вода перекрыта


## 21. Strategy — алгоритм подставляется

**Легенда:** Посылка одна. Курьером или почтой. Меняешь способ — посылка та же, доставка разная.

**Зачем нужен:** Один результат разными способами. Объект хранит «стратегию», делегирует ей. Сортировка, оплата, доставка.

In [23]:
%%writefile strategy.cpp
/* ЛЕГЕНДА: Посылка одна. Курьером или почтой. Способ разный, можно менять на лету. СУТЬ: разные алгоритмы под одним интерфейсом. */

#include <iostream>
using namespace std;

// Стратегия — способ доставки (интерфейс)
class Delivery {
public:
    virtual void deliver(const string& parcel) = 0;
    virtual ~Delivery() = default;
};

// Конкретные стратегии
class Courier : public Delivery {
    void deliver(const string& p) override { cout << "Курьер: " << p << endl; }
};
class Post : public Delivery {
    void deliver(const string& p) override { cout << "Почта: " << p << endl; }
};

// Контекст — использует стратегию. Можно менять setMethod() в любой момент
class Sender {
    Delivery* method;  // текущий способ доставки
public:
    void setMethod(Delivery* d) { method = d; }
    void send(const string& p) {
        method->deliver(p);  // делегируем стратегии
    }
};

int main() {
    Sender s;
    s.setMethod(new Courier());
    s.send("Коробка");       // курьером
    s.setMethod(new Post()); // сменили стратегию
    s.send("Коробка");       // почтой
}

Writing strategy.cpp


In [24]:
!g++ -std=c++17 -o out strategy.cpp && ./out

Курьер: Коробка
Почта: Коробка


## 22. Template Method — скелет + шаги

**Легенда:** Рецепт. Общая схема: подготовить, готовить, подать. Суп и яичница — по-разному.

**Зачем нужен:** Общий алгоритм с вариациями. Базовый класс задаёт порядок, подклассы — конкретные шаги.

In [25]:
%%writefile template_method.cpp
/* ЛЕГЕНДА: Рецепт — подготовить, готовить, подать. Суп и яичница по-разному. СУТЬ: базовый класс задаёт скелет, подклассы — конкретные шаги. */

#include <iostream>
using namespace std;

// Базовый класс — скелет алгоритма (template method)
// cook() — шаблонный метод: вызывает шаги в фиксированном порядке
class Recipe {
public:
    void cook() {
        prepare();   // шаг 1 — каждый свой (виртуальный)
        cookStep();  // шаг 2 — каждый свой
        serve();     // шаг 3 — общий для всех
    }
    virtual ~Recipe() = default;
protected:
    virtual void prepare() = 0;   // абстрактные — наследники реализуют
    virtual void cookStep() = 0;
    void serve() { cout << "Подать\n"; }  // общий метод, не виртуальный
};

// Суп — свои шаги
class Soup : public Recipe {
    void prepare() override { cout << "Режем овощи\n"; }
    void cookStep() override { cout << "Варим\n"; }
};

// Яичница — другие шаги
class Omelet : public Recipe {
    void prepare() override { cout << "Взбиваем яйца\n"; }
    void cookStep() override { cout << "Жарим\n"; }
};

int main() {
    Soup s;
    s.cook();
    Omelet o;
    o.cook();
}

Writing template_method.cpp


In [26]:
!g++ -std=c++17 -o out template_method.cpp && ./out

Режем овощи
Варим
Подать
Взбиваем яйца
Жарим
Подать


## 23. Visitor — операция снаружи

**Легенда:** Магазин. Обходишь полки. Задача «посчитать сумму» — один обход. «Проверить наличие» — тот же обход, другая операция.

**Зачем нужен:** Новая операция над структурой без изменения элементов. Visitor — объект с `visit(Product*)`, `visit(Shelf*)`. Элемент вызывает `accept(visitor)`.

In [27]:
%%writefile visitor.cpp
/* ЛЕГЕНДА: Магазин — обходишь полки, считаешь сумму. Новая операция без изменения Product/Shelf. СУТЬ: операция «снаружи» от структуры. */

#include <iostream>
#include <vector>
using namespace std;

class Product;
class Shelf;

// Визитёр — операция. Умеет обрабатывать Product и Shelf по-разному
class ShopVisitor {
public:
    virtual void visit(Product* p) = 0;
    virtual void visit(Shelf* s) = 0;
    virtual ~ShopVisitor() = default;
};

// Элемент — принимает визитёра (double dispatch: element->accept(visitor) -> visitor->visit(this))
class StoreItem {
public:
    virtual void accept(ShopVisitor* v) = 0;
    virtual ~StoreItem() = default;
};

// Товар — вызывает v->visit(this), передавая себя визитёру
class Product : public StoreItem {
    string name;
    int price;
public:
    Product(const string& n, int p) : name(n), price(p) {}
    void accept(ShopVisitor* v) override { v->visit(this); }  // this — указатель на себя
    string getName() { return name; }
    int getPrice() { return price; }
};

// Полка — принимает визитёра и передаёт ему детей (рекурсия по дереву)
class Shelf : public StoreItem {
    string name;
    vector<StoreItem*> items;
public:
    Shelf(const string& n) : name(n) {}
    void add(StoreItem* i) { items.push_back(i); }
    void accept(ShopVisitor* v) override {
        v->visit(this);
        for (auto i : items)
            i->accept(v);  // визитёр обходит детей
    }
};

// Конкретный визитёр — считает сумму цен
class PriceVisitor : public ShopVisitor {
    int total = 0;
public:
    void visit(Product* p) override { total += p->getPrice(); }
    void visit(Shelf*) override {}  // полка сама по себе не имеет цены
    int getTotal() { return total; }
};

int main() {
    Shelf sh("Полка");
    sh.add(new Product("Хлеб", 50));
    sh.add(new Product("Молоко", 80));
    PriceVisitor pv;
    sh.accept(&pv);  // обходим всё дерево, считаем
    cout << "Сумма: " << pv.getTotal() << endl;
}

Writing visitor.cpp


In [28]:
!g++ -std=c++17 -o out visitor.cpp && ./out

Сумма: 130
